## Model interpretation

### Investigation on the damping effect of differnt air temperature amplitudes in the soil temperature model

In [ ]:
import pandas as pd
from pandas import array

import numpy as np
import time
import math
%matplotlib widget
import matplotlib.pyplot as plt
import matplotlib.dates

from numpy.lib import stride_tricks

from keras.models import Model
from keras.layers import Input, LSTM, Dense, LSTMCell, RNN, Bidirectional, Concatenate, GRU, RepeatVector, TimeDistributed, Dropout, Concatenate, BatchNormalization
from keras.optimizers import Adam
from keras.utils.vis_utils import plot_model
from keras import callbacks, layers

import tensorflow as tf

from scipy import signal

#from tensorflow import keras
#from tensorflow.keras import layers
print(np.version.version)
print(tf.__version__)

### Load soil temperature model

In [2]:
#new_model = tf.keras.models.load_model('') #  fill in the directory to the model

### Read data

In [4]:
windowSize=240

data_row = pd.read_csv("data_st_sm_11_years.csv")

at = np.squeeze(data_row["AirTempHourely"].to_numpy())
pr = np.squeeze(data_row["PrecHourely"].to_numpy())
st05 = np.squeeze(data_row["SoilTempHourely"].to_numpy()) 
sm05 = np.squeeze(data_row["SoilMoiHourely"].to_numpy()) 


### Data normalisation

In [5]:
def create_norm_seq(seq,seq_norm_factor):
    seq_norm= (seq-np.min(seq_norm_factor))/(np.max(seq_norm_factor)-np.min(seq_norm_factor))
    return seq_norm

def create_re_norm_seq(seq,seq_norm_factor):
    re_norm_seq = (seq*(max(seq_norm_factor)-min(seq_norm_factor)))+min(seq_norm_factor)#(seq-np.min(seq))/(np.max(seq)-np.min(seq))
    return re_norm_seq


### Function for model prediciton

In [6]:

def predict_sequence(input_array_1,input_array_2):
    seqNum=np.floor(len(input_array_1)/windowSize)

    data2a = input_array_1[:int(seqNum)*windowSize]
    x1=np.split(data2a,seqNum)
    x1=np.stack(x1,axis=1)

    data2b = input_array_2[:int(seqNum)*windowSize]
    x2=np.split(data2b,seqNum)
    x2=np.stack(x2,axis=1)

    x1 = np.transpose(x1)
    x1 = x1.reshape(x1.shape[0],x1.shape[1],1)

    x2 = np.transpose(x2)
    x2 = x2.reshape(x2.shape[0],x2.shape[1],1)

    year_predict = new_model.predict([x1,x2],batch_size=1)

    year_predict = np.squeeze(year_predict)
    year_predict = year_predict.ravel()

    return year_predict


### Specifications of the simplified synthetic air temperature sequence

In [7]:
Fs = 2400 #8760
f = 1/24
sample = 2400#8760
period = 24/Fs
x = np.arange(sample)
rain_zero = np.zeros(Fs)

### One single example of the damping effect in the model to investigate the temporal shift of the sequences

In [ ]:
temp_sine= 2.5*np.sin(Fs*2 * np.pi * f * x/Fs)+10
rain_zero = np.zeros(Fs)

temp_sine_norm = create_norm_seq(temp_sine,at)

soil_sine_norm_t = predict_sequence(temp_sine_norm,rain_zero)

soil_sine_t = create_re_norm_seq(soil_sine_norm_t,st05)

plt.figure(figsize=(10,5))
plt.margins(x=0)
plt.subplots_adjust(left=0.2, right=0.9, top=0.9, bottom=0.2)
plt.plot(temp_sine[1000:1240],label="Input: Synthetic air temperature",color="pink",alpha=0.7,zorder=1)
plt.plot(soil_sine_t[1000:1240],label="Prediction: Soil temperature",color="red")
plt.ylabel("Soil / air temperature [°C]",fontsize=13)
plt.tick_params(labelsize= 13)
plt.xlabel("Hours [h]",fontsize=13)
plt.legend(prop={'size': 13},loc='lower left')
plt.ylim(5,15)
plt.show()

### Model predictions based on air temperature with different amplitudes

In [ ]:

ampArray = np.arange(0.2, 7, 0.2)

amplitudeList = []
lagList = []
airTempList = []
soilTempList = []


for amp in ampArray:
    
    air_temperature = amp*np.sin(Fs*2 * np.pi * f * x/Fs)+10#2.5*np.sin(freq*2 * np.pi * f * x/Fs)#10
    air_temperature_norm = create_norm_seq(air_temperature,at)
    soil_temperature_norm = predict_sequence(air_temperature_norm, rain_zero)
    soil_temperature_norm = predict_sequence(air_temperature_norm, rain_zero)
    soil_temperature_norm = predict_sequence(air_temperature_norm, rain_zero)
    soil_temperature = create_re_norm_seq(soil_temperature_norm,st05)

    airTempList.append(air_temperature)
    soilTempList.append(soil_temperature)

    amplitude = max(soil_temperature[500:2500])-min(soil_temperature[500:2500])
    amplitudeList.append(amplitude)

    correlation = signal.correlate(air_temperature, soil_temperature, mode="full")
    lags = signal.correlation_lags(air_temperature.size, soil_temperature.size, mode="full")
    lag = lags[np.argmax(correlation)]
    lagList.append(lag)
print(amplitudeList)

amplitudeArray_10=np.array(amplitudeList)
lagArray_10=np.array(lagList)

### Load data again

In [11]:
import pandas as pd

df = pd.read_csv("data_st_sm_11_years.csv")

In [13]:
from scipy.fft import fft, fftfreq
import numpy as np

### Create air temperature and soil temperature sequence

In [14]:
at_seq = df['AirTempHourely'].to_numpy()
st_seq = df['SoilTempHourely'].to_numpy()

### Extract corresponding amplitudes of air temperature and soil temperature from the measurement data

In [ ]:
splits = int(len(at_seq)/168)-1

list_amp=[]
amp_damping_freq=[]
for i in range(splits):
    st = st_seq[i*168:(i+1)*168]
    at = at_seq[i*168:(i+1)*168]
    st_mean = np.mean(st)
    at_mean = np.mean(at)
    N = int(len(at))
    dt = 1 #hours

    yf = np.abs(fft(at)[:N//2])
    xf = fftfreq(N, dt)[:N//2]
    yf_st = np.abs(fft(st))[:N//2]

    daily_amp_at = yf[np.argmin(np.abs((1/xf)-24))]*2.0/N
    daily_amp_st = yf_st[np.argmin(np.abs((1/xf)-24))]*2.0/N
    list_amp.append([at_mean, st_mean, daily_amp_at, daily_amp_st])
    amp_damping_freq.append(yf_st/yf)
        

In [17]:
amp_damping_freq_array= np.array(amp_damping_freq)

In [18]:
amp_array =np.array(list_amp)
amp_array

amp_df = pd.DataFrame()
amp_df['at_mean'] = amp_array[:,0]
amp_df['st_mean'] = amp_array[:,1]
amp_df['daily_amp_at'] = amp_array[:,2]
amp_df['daily_amp_st'] = amp_array[:,3]

In [19]:
amp_df_filtered = amp_df.loc[(amp_df['at_mean'] > 6)&(amp_df['at_mean'] < 15 ) ]

### Damping effect of the soil temperature model 

In [ ]:

plt.figure(figsize=(5,5))
plt.scatter(amp_df_filtered['daily_amp_at'], amp_df_filtered['daily_amp_st'],color="indianred",alpha=0.5,linewidth=0,s=70,label = "In situ data")
plt.plot(ampArray,amplitudeArray_10,'g^',color="black",markersize=7,label="Model prediction")
plt.xlabel("Amplitude Air Temperature [°C]",fontsize = 13)
plt.ylabel("Amplitude Soil Temperature [°C]",fontsize = 13)
plt.xticks(fontsize = 13)
plt.yticks(fontsize = 13)
plt.legend(fontsize = 13)
plt.xlim(-0.2,5.1)
plt.ylim(-0.05,1)
plt.show()

### Creating plot for publication

In [ ]:
fig, ax = plt.subplots(1,2,figsize=(15,5),width_ratios=[2, 1])
fig.subplots_adjust(bottom=0.17,right=0.9,wspace=0.2)

ax[0].margins(x=0)
ax[0].plot(temp_sine[1000:1240],label="Input: Synthetic air temperature",color="pink",alpha=0.9,zorder=1)
ax[0].plot(soil_sine_t[1000:1240],label="Prediction: Soil temperature",color="red")
ax[0].set_ylabel("Soil / air temperature [°C]",fontsize=13)
ax[0].tick_params(labelsize= 13)
ax[0].set_xlabel("Hours [h]",fontsize=13)
ax[0].legend(prop={'size': 13},loc='upper left')
ax[0].set_ylim(5,15)

ax[1].scatter(amp_df_filtered['daily_amp_at'], amp_df_filtered['daily_amp_st'],color="indianred",alpha=0.5,linewidth=0,s=70,label = "Measurement data")
ax[1].plot(ampArray,amplitudeArray_10,'g^',color="black",markersize=7,label="Model prediction")
ax[1].set_xlabel("Amplitude Air Temperature [°C]",fontsize = 13)
ax[1].set_ylabel("Amplitude Soil Temperature [°C]",fontsize = 13)
ax[1].tick_params(labelsize = 13)
ax[1].legend(fontsize = 13)
ax[1].set_xlim(-0.2,5.1)
ax[1].set_ylim(-0.1,0.95)

ax[0].text(-19, 14.9, "(a)", fontsize = 21)#weight='bold'
ax[1].text(-1.05, 0.94, "(b)", fontsize = 21)



plt.savefig('Fig5.pdf',bbox_inches='tight')
